In [ ]:
import networkx as nx
%matplotlib inline

# Chapter 2 Tutorial

多くの練習問題の後に `assert` 文を含むブロックがあることに注意されたい。この `assert` 文の前にセットアップコードがある場合がある。これはあなたが正しい道にいるというフィードバックを与えるために設置されている。もし `AssertionError` を受け取ったら、おそらくどこかに間違いがあるということだ。

Contents:

1. 経路（Paths）
2. 連結成分（Connected components）
3. 有向路（Directed paths & components）
4. Dataset: US air traffic network

## 1. 経路

無向ネットワークという簡単な例から始めよう。

In [ ]:
G = nx.Graph()

G.add_nodes_from([1,2,3,4])

G.add_edges_from([(1,2),(2,3),(1,3),(1,4)])

nx.draw(G, with_labels=True)

ネットワーク内の経路（*path*）とは、二つのノードをつなぐエッジの集合（エッジに沿ってたどることができる道筋）である。 この単純な例では、ノード3と4を結ぶパスが少なくとも1つ存在することが簡単にわかる。このことは NetworkX で確認できる。

In [ ]:
nx.has_path(G, 3, 4)

二つのノードの間に複数の経路がある場合がある。再びノード3と4を考えたとき、そのような「単純な」経路が二つある。

In [ ]:
list(nx.all_simple_paths(G, 3, 4))

単純な経路（*simple path*）とは、サイクルのない経路である。もしサイクルを許せば、いつでも好きなだけサイクルを回れるので、無限の経路が存在することになる。

私たちはしばしば、最短経路（*shortest path*）に最も興味を持つ。重みのないネットワークでは、最短経路は最も少ないエッジを持つ経路である。ノード3とノード4の間の2つの単純な経路のうち、一方が他方より短いことがわかる。この最短経路は、NetworkX の関数を使うことで得ることができる。

In [ ]:
nx.shortest_path(G, 3, 4)

経路長（*path length*）にのみ関心がある時は、次の関数が用意されている。

In [ ]:
nx.shortest_path_length(G, 3, 4)

ここで、経路長が経路に含まれるノードの数ではなく、**エッジ**の数として定義されていることに注意しよう。つまり、ノード $u$ と $v$ に対して、

    nx.shortest_path_length(G, u, v) == len(nx.shortest_path(G, u, v)) - 1



## 2. 連結成分（*connected components*）

上の単純なネットワークでは、どのノードのペアに対してもそれらをつなぐ経路が存在する。これは連結グラフ（*connected graph*）の定義である。所与のグラフについて、次のようにしてこの連結性を確認することができる。

In [ ]:
nx.is_connected(G)

すべてのグラフが連結しているわけではない。

In [ ]:
G = nx.Graph()

G.add_cycle((1,2,3))
G.add_edge(4,5)

nx.draw(G, with_labels=True)

In [ ]:
nx.is_connected(G)

ノード間に経路が存在しないのに経路を確かめようとすると、NetworkX はエラーを返してくる。

In [ ]:
nx.shortest_path(G ,3, 5)

このグラフにおいて、二つの連結成分を視覚的に確認できるが、コードで検証してみよう。

In [ ]:
nx.number_connected_components(G)

関数 `nx.connected_components()` はグラフを受け取り、それぞれの連結成分に含まれるノードの集合を要素に持つリストを返す。次のリストに格納された二つの集合が、上で描画したグラフにある二つの連結成分に対応していることを確認しよう。

In [ ]:
list(nx.connected_components(g))

Pythonの集合に馴染みがない方のために説明すると、集合とは重複のない要素のコレクションのことである。ノード名は一意なので、ノード名を収集するのに便利なデータ型だ。リストのような他のコレクションと同様に、 `len` 関数で集合内の要素数を取得できる。

In [ ]:
components = list(nx.connected_components(G))
len(components[0])

私たちはしばしば、ネットワークのコア（*core*）とも呼ばれる最大連結成分に注目する。最大連結成分を得るには、Pythonの組み込みの `max()` 関数を使用することができる。デフォルトでは、Pythonの `max()` 関数は辞書順（すなわちアルファベット順）でソートするが、これはここでは役に立たない。ここでは、大きさの順にソートされたときの最大連結成分が欲しいので、キー関数に `len` を渡す。

In [ ]:
max(nx.connected_components(G), key=len)

ノード名のリストを得るだけで十分な場合もあるが、最大連結成分からなる実際のサブグラフが必要な場合もある。これを得るための1つの方法は、ノード名のリストを `G.subgraph()` 関数に渡すことです。

In [ ]:
core_nodes = max(nx.connected_components(G), key=len)
core = G.subgraph(core_nodes)

nx.draw(core, with_labels=True)

タブのコード補完機能を使用している人は、`nx.connected_component_subgraphs()`関数にも気が付くはずだ。これもコアサブグラフを得るために使用できるが、最大連結成分にしか興味がない場合は、先に紹介した方法の方が効率的である。